In [1]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/huiqy/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def batch_canny(images):
    canny_list = []
    for i in tqdm(range(len(images))):
        img = (images.astype(np.float32)* 255)[i, :, :, :].astype(np.uint8)
        canny_superposition = np.zeros(shape=(img.shape[0],img.shape[1]),dtype=np.uint16)
        for channel in range(3):
            img_ch = img[:,:,channel]
            canny_ch = cv2.Canny(img_ch, 60, 150)
            canny_superposition =  canny_superposition + (canny_ch).astype(np.uint16)
        canny_superposition = cv2.normalize(canny_superposition, canny_superposition,0,255,cv2.NORM_MINMAX)
        canny_list.append(np.array((canny_superposition).astype(np.uint8)).reshape([img.shape[0],img.shape[1],1]))
    canny_list = np.array(canny_list,dtype = float32)/255.0
    return canny_list


In [3]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

def load_flower_data():
    # grab the list of images that we'll be describing
    print("[INFO] handling images...")
    TRAIN_ORIGINAL_DIR = '../train/'
    TRAIN_SUB_DIR = '../subsample/'
    TRAIN_GAN = '../../image_gan/'
    TEST_DIR = '../../test/'

    # use this for full dataset
#     train_images_original = [TRAIN_ORIGINAL_DIR + i for i in os.listdir(TRAIN_ORIGINAL_DIR)]
#     train_images_sub = [TRAIN_SUB_DIR + i for i in os.listdir(TRAIN_SUB_DIR)]
#     train_images = train_images_original + train_images_sub
    train_images_gan = [TRAIN_GAN + i for i in os.listdir(TRAIN_GAN)]
    test_images = [TEST_DIR + i for i in os.listdir(TEST_DIR)]
    
    train_images = train_images_gan
    
    train_images.sort(key=natural_keys)
    test_images.sort(key=natural_keys)

    # initialize the features matrix and labels list
    trainImage = []
    trainLabels = []
    testImage = []
    testLabels = []

    # loop over the input images
    for (i, imagePath) in enumerate(train_images):
        # extract the class label
        # get the labels from the name of the images by extract the string before "_"
        label = imagePath.split(os.path.sep)[-1].split("_")[0]

        # read and resize image
        img = cv2.imread(imagePath)
        img = cv2.resize(img, (128,128))

        # add the messages we got to features and labels matricies
        trainImage.append(img)
        trainLabels.append(label)

        # show an update every 100 images until the last image
        if i > 0 and ((i + 1) % 1000 == 0 or i == len(train_images) - 1):
            print("[INFO] processed {}/{}".format(i + 1, len(train_images)))
            
      # loop over the input images
    for (i, imagePath) in enumerate(test_images):
        # extract the class label
        # our images were named as labels.image_number.format
        # get the labels from the name of the images by extract the string before "."
        label = imagePath.split(os.path.sep)[-1].split("_")[0]

        # extract CNN features in the image
        img = cv2.imread(imagePath)
        img = cv2.resize(img, (128,128))

        # add the messages we got to features and labels matricies
        testImage.append(img)
        testLabels.append(label)

        # show an update every 100 images until the last image
        if i > 0 and ((i + 1) % 1000 == 0 or i == len(test_images) - 1):
            print("[INFO] processed {}/{}".format(i + 1, len(test_images)))


    trainImage = np.array(trainImage,dtype = float32)
    trainLabels = np.array(trainLabels)
    testImage = np.array(testImage,dtype = float32)
    testLabels = np.array(testLabels)
    print (trainImage.shape)
    
    trainImage = trainImage.astype(np.float32) / 255.0
    testImage = testImage.astype(np.float32) / 255.0
    
    le = preprocessing.LabelEncoder()
    le.fit(trainLabels)
    list(le.classes_)
    trainLabels = le.transform(trainLabels) 
    testLabels = le.transform(testLabels) 
    
    return trainImage, trainLabels, testImage, testLabels

In [4]:
trainImage, trainLabels, testImage, testLabels = load_flower_data()

trainImage_canny = batch_canny(trainImage)
testImage_canny = batch_canny(testImage)

nb_classes = 2

# Convert class vectors to binary class matrices.
trainLabels = keras.utils.to_categorical(trainLabels, nb_classes)
print (trainLabels)
testLabels = keras.utils.to_categorical(testLabels, nb_classes)
print (testLabels)
print (testLabels.shape)

np.save('../trainImage.npy', trainImage)
np.save('../trainLabels.npy', trainLabels)
np.save('../testImage.npy', testImage)
np.save('../testLabels.npy', testLabels)
np.save('../trainImage_canny.npy', trainImage_canny)
np.save('../testImage_canny.npy', testImage_canny)

print("[INFO] trainImage matrix: {:.2f}MB".format(
    (trainImage.nbytes) / (1024 * 1000.0)))
print("[INFO] trainLabels matrix: {:.4f}MB".format(
    (trainLabels.nbytes) / (1024 * 1000.0)))
print("[INFO] testImage matrix: {:.2f}MB".format(
    (testImage.nbytes) / (1024 * 1000.0)))
print("[INFO] testLabels matrix: {:.4f}MB".format(
    (testLabels.nbytes) / (1024 * 1000.0)))
print("[INFO] trainImage_canny matrix: {:.2f}MB".format(
    (trainImage_canny.nbytes) / (1024 * 1000.0)))
print("[INFO] testImage_canny matrix: {:.4f}MB".format(
    (testImage_canny.nbytes) / (1024 * 1000.0)))

[INFO] handling images...
[INFO] processed 1000/2000
[INFO] processed 2000/2000
[INFO] processed 158/158
(2000, 128, 128, 3)


100%|██████████| 158/158 [00:03<00:00, 48.80it/s]


[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0.

In [5]:
class CNN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, log_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.log_dir = log_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size

        # parameters
        self.input_height = 128
        self.input_width = 128
        self.c_dim = 3  # color dimension
        self.nb_class = 2
        
        # number of convolutional filters to use  
        self.nb_CNN = [32, 64, 64, 64, 128]  
        # number of dense filters to use  
        self.nb_Dense = [256] 
        # size of pooling area for max pooling  
        self.pool_size = (2, 2)  
        # convolution kernel size  
        self.kernel_size = (3, 3)
        self.batch_normalization_control = True
        
        # name for checkpoint
        self.model_name = 'CNN_CANNY_C%d_D%d_Kernel(%d,%d)_lrdecay0.95' % (len(self.nb_CNN), len(self.nb_Dense),
                                                          self.kernel_size[0], self.kernel_size[1])    

        # train
        #设置一个全局的计数器
        self.global_step = tf.Variable(0, trainable=False)
        self.lr = tf.train.exponential_decay(0.001, 
                                             global_step=self.global_step, 
                                             decay_steps=10, 
                                             decay_rate=0.95, 
                                             staircase=True)
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300

        # test

        #load_flower_data
        self.train_x = np.load('../trainImage.npy')
        self.train_y= np.load('../trainLabels.npy')
        self.test_x = np.load('../testImage.npy')
        self.test_y= np.load('../testLabels.npy')
        self.train_x_canny = np.load('../trainImage_canny.npy')
        self.test_x_canny = np.load('../testImage_canny.npy')
        
        #记录
        self.train_hist = {}
        self.train_hist['losses'] = []
        self.train_hist['accuracy'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches_train = len(self.train_x) // self.batch_size
        self.num_batches_test= len(self.test_x) // self.batch_size

    def cnn_model(self, x, x_canny, keep_prob, is_training=True, reuse=False):
        with tf.variable_scope("cnn", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            B = tf.constant_initializer(0.0)
        
            print("CNN:x",x.get_shape()) # 128, 128, 3 
            print("CNN:x_canny",x_canny.get_shape()) # 128, 128, 1
            
            #输入x,卷积核为3*3 输出维度为32
            net1_1 = tf.layers.conv2d(inputs = x,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_1'               # 命名用于获取变量
                                    )
            print("CNN:",net1_1.get_shape())
            
            #输入x,卷积核为3*3 输出维度为32
            net1_2 = tf.layers.conv2d(inputs = x_canny,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_2'               # 命名用于获取变量
                                    )
            print("CNN:",net1_2.get_shape())

            #把数据和边缘进行连接
            net = tf.concat([net1_1, net1_2], 3)
            net = tf.layers.batch_normalization(net, training=is_training)
            net = tf.nn.relu(net, name = 'relu_conv_1')
            print("CNN:",net.get_shape())
            net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_1'
                                             )
            
            for i in range(2,len(self.nb_CNN)+1):
                net = tf.layers.conv2d(inputs = net,                 # 输入,
                                       filters = self.nb_CNN[i-1],      # 卷积核个数,
                                       kernel_size = self.kernel_size,          # 卷积核尺寸
                                       strides = (1, 1),
                                       padding = 'same',              # padding方法
                                       kernel_initializer = W,
                                       bias_initializer = B,
                                       kernel_regularizer = None,
                                       bias_regularizer = None,
                                       activity_regularizer = None,
                                       name = 'conv_'+ str(i)        # 命名用于获取变量
                                       )
                print("CNN:",net.get_shape())
                if self.batch_normalization_control:
                    net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_conv_' + str(i))
                net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_' + str(i)
                                             )
                print("CNN:",net.get_shape())
            
            #flatten
            net = tf.reshape(net, [-1, int(net.get_shape()[1]*net.get_shape()[2]*net.get_shape()[3])],name='flatten')
            print("CNN:",net.get_shape())
            
            #dense layer
            for i in range(1,len(self.nb_Dense)+1):
                net = tf.layers.dense(inputs = net,
                                    units = self.nb_Dense[i-1],
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_' + str(i)
                                    )
#                 net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_dense_' + str(i))
                net = tf.layers.dropout(inputs = net,
                                        rate=keep_prob,
                                        noise_shape=None,
                                        seed=None,
                                        training = is_training,
                                        name= 'dropout_dense_' + str(i)
                                        )
            #output
            logit = tf.layers.dense(inputs = net,
                                    units = self.nb_class,
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_output'
                                    )
            out_logit = tf.nn.softmax(logit, name="softmax")
            print("CNN:out_logit",out_logit.get_shape())
            print("------------------------")    

            return out_logit, logit


    def build_model(self):

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.c_dim], 
                                name='x_image')
        
        self.x_canny = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, 1], 
                                name='x_canny')

        self.y = tf.placeholder(tf.float32, shape=[self.batch_size, self.nb_class], name='y_label')
        
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.add_global = self.global_step.assign_add(1)

        """ Loss Function """

        # output of cnn_model
        self.out_logit, self.logit = self.cnn_model(self.x, self.x_canny, self.keep_prob, is_training=True, reuse=False)
        
        self.loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y,
                                                                                         logits =self.logit))
        
        """ Training """
        # trainable variables into a group
        tf_vars = tf.trainable_variables()
        cnn_vars = [var for var in tf_vars if var.name.startswith('cnn')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.cnn_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.loss_cross_entropy,
                                                                                        var_list=cnn_vars)
        """" Testing """
        # for test
        # output of cnn_model
        self.out_logit_test, self.logit_test = self.cnn_model(self.x, self.x_canny, self.keep_prob, is_training=False, reuse=True)
        self.correct_prediction = tf.equal(tf.argmax(self.logit_test, 1), tf.argmax(self.y, 1))
        self.predict = tf.argmax(self.logit_test, 1)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        """ Summary """
        self.loss_sum = tf.summary.scalar("loss", self.loss_cross_entropy)


    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()
#         sess.run(tf.global_variables_initializer())

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep = self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + self.model_name+'_train_hist.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            CNN_losses = []
  
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.train_x.shape[0]), self.train_x.shape[0])
            shuffled_set = self.train_x[shuffle_idxs]
            shuffled_set_canny = self.train_x_canny[shuffle_idxs]
            shuffled_label = self.train_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches_train):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_canny = shuffled_set_canny[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_y = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                

                # update D network
                _, summary_str, cnn_loss = self.sess.run([self.cnn_optim, self.loss_sum, self.loss_cross_entropy],
                                               feed_dict={self.x: batch_x,
                                                          self.x_canny: batch_x_canny,
                                                          self.y: batch_y,
                                                          self.keep_prob: 0.5}
                                                      )
                self.writer.add_summary(summary_str, counter)

                CNN_losses.append(cnn_loss)

                # display training status
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, loss: %.8f" % (epoch_loop, idx, self.num_batches_train, 
                                                                          time.time() - start_time, cnn_loss))

            # After an epoch
            # Evaluates accuracy on test set
                    # Evaluates accuracy on test set
            test_accuracy_list = []
            for idx_test in range(self.num_batches_test):
                batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_canny_test = self.test_x_canny[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                    self.x_canny: batch_x_canny_test,
                                                                    self.y: batch_y_tes,
                                                                    self.keep_prob: 1.0})
                test_accuracy_list.append(accuracy)
            test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        
            #update learning rate
            _, rate = sess.run([self.add_global, self.lr])
            
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss: %.8f acc: %.5f'% (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                    np.mean(CNN_losses), test_accuracy))
            self.train_hist['losses'].append(np.mean(CNN_losses))
            self.train_hist['accuracy'].append( test_accuracy)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)
            
            # save trainhist for train
            f = open(self.trainhist_dir + '/' + self.model_name + '_train_hist.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' 
                                 + self.model_name + '_train_hist.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
        
        """test after train"""
        best_acc = max(self.train_hist['accuracy'])
        beat_epoch = self.train_hist['accuracy'].index(best_acc) + 1
        print (" [*] Best Epoch: ", beat_epoch, ", Accuracy: ", best_acc)
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(beat_epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

                # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_canny_test = self.test_x_canny[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_canny: batch_x_canny_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Best Epoch:", beat_epoch, ", accuracy: ",test_accuracy, "!")

    def test(self, epoch):
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_canny_test = self.test_x_canny[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_canny: batch_x_canny_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Epoch:", epoch, ", accuracy: ",test_accuracy, "!")
        
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True) 

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['losses'])+1)

        y1 = hist['losses']
        y2 = hist['accuracy']
        
        fig, ax1 = plt.subplots()
                            
        ax2 = ax1.twinx()  

        ax1.plot(x, y1, 'b')
        ax2.plot(x, y2, 'r')
                            
        ax1.set_xlabel('Epoch')
                            
        ax1.set_ylabel('CNN_loss')    
        ax2.set_ylabel('accuracy')

        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
   

In [6]:
gan_type = 'CNN'
dataset = '4_Flowers'
epoch = 80
batch_size = 100
checkpoint_dir = 'checkpoint'
log_dir = 'logs'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --log_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    CNN = CNN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                log_dir=log_dir, trainhist_dir=trainhist_dir)

    # build graph
    CNN.build_model()

    # show network architecture
    CNN.show_all_variables()

    # launch the graph in a session
    CNN.train()
    
#     CNN.test(epoch)
        
sess.close()
# Avg per epoch ptime: 11.34, total 80 epochs ptime: 967.46
#  [*] Training finished!
#  [*] Best Epoch:  68 , Accuracy:  0.7400000095367432
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_CANNY_C5_D1_Kernel(3,3)_lrdecay0.95/CNN_CANNY_C5_D1_Kernel(3,3)_lrdecay0.95-68
#  [*] Finished testing Best Epoch: 68 , accuracy:  0.7400000095367432 !

# Avg per epoch ptime: 5.46, total 80 epochs ptime: 484.56
#  [*] Training finished!
#  [*] Best Epoch:  9 , Accuracy:  0.800000011920929
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_CANNY_C5_D1_Kernel(3,3)_lrdecay0.95/CNN_CANNY_C5_D1_Kernel(3,3)_lrdecay0.95-9
#  [*] Finished testing Best Epoch: 9 , accuracy:  0.800000011920929 !

CNN:x (100, 128, 128, 3)
CNN:x_canny (100, 128, 128, 1)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 64)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
CNN:x (100, 128, 128, 3)
CNN:x_canny (100, 128, 128, 1)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 64)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
---------
Variables: name (type shape) [size]
---------
cnn/conv_1_1/kernel:0 (float32_ref 3x3x3x32) [864, bytes: 3456]
cnn/conv_1_1/bias:0 (float32_ref 32) [32, bytes: 128]
cnn/conv_1_2/kernel:0 (float32_ref 3x3x1x32) [288, bytes: 1152]
cnn/conv_1

Epoch: [ 5] [  17/  20] time: 31.2190, loss: 0.01636663
Epoch: [ 5] [  18/  20] time: 31.4762, loss: 0.03469323
Epoch: [ 5] [  19/  20] time: 31.7186, loss: 0.04057778
[5/80] - ptime: 5.3529 loss: 0.03917817 acc: 0.79000
Epoch: [ 6] [   0/  20] time: 33.1316, loss: 0.00800829
Epoch: [ 6] [   1/  20] time: 33.3801, loss: 0.01926140
Epoch: [ 6] [   2/  20] time: 33.6309, loss: 0.04586631
Epoch: [ 6] [   3/  20] time: 33.8779, loss: 0.04716146
Epoch: [ 6] [   4/  20] time: 34.1215, loss: 0.01610855
Epoch: [ 6] [   5/  20] time: 34.3693, loss: 0.01079348
Epoch: [ 6] [   6/  20] time: 34.6199, loss: 0.02488069
Epoch: [ 6] [   7/  20] time: 34.8680, loss: 0.01360999
Epoch: [ 6] [   8/  20] time: 35.1126, loss: 0.03039996
Epoch: [ 6] [   9/  20] time: 35.3636, loss: 0.18675193
Epoch: [ 6] [  10/  20] time: 35.6169, loss: 0.07736140
Epoch: [ 6] [  11/  20] time: 35.8628, loss: 0.02892289
Epoch: [ 6] [  12/  20] time: 36.1042, loss: 0.02623400
Epoch: [ 6] [  13/  20] time: 36.3586, loss: 0.0548

Epoch: [12] [  17/  20] time: 73.2993, loss: 0.01751679
Epoch: [12] [  18/  20] time: 73.5474, loss: 0.01105223
Epoch: [12] [  19/  20] time: 73.8081, loss: 0.01405674
[12/80] - ptime: 5.4578 loss: 0.01647525 acc: 0.66000
Epoch: [13] [   0/  20] time: 75.0136, loss: 0.00556115
Epoch: [13] [   1/  20] time: 75.2570, loss: 0.01049671
Epoch: [13] [   2/  20] time: 75.5049, loss: 0.02753647
Epoch: [13] [   3/  20] time: 75.7622, loss: 0.01194715
Epoch: [13] [   4/  20] time: 76.0250, loss: 0.00166341
Epoch: [13] [   5/  20] time: 76.2711, loss: 0.01247774
Epoch: [13] [   6/  20] time: 76.5185, loss: 0.01837321
Epoch: [13] [   7/  20] time: 76.7765, loss: 0.00486453
Epoch: [13] [   8/  20] time: 77.0299, loss: 0.04732073
Epoch: [13] [   9/  20] time: 77.2798, loss: 0.00599907
Epoch: [13] [  10/  20] time: 77.5304, loss: 0.00404183
Epoch: [13] [  11/  20] time: 77.7886, loss: 0.00829322
Epoch: [13] [  12/  20] time: 78.0522, loss: 0.01808547
Epoch: [13] [  13/  20] time: 78.2981, loss: 0.005

Epoch: [19] [  16/  20] time: 115.2434, loss: 0.00090155
Epoch: [19] [  17/  20] time: 115.4944, loss: 0.01603347
Epoch: [19] [  18/  20] time: 115.7401, loss: 0.02258901
Epoch: [19] [  19/  20] time: 115.9928, loss: 0.00924097
[19/80] - ptime: 5.5552 loss: 0.00944703 acc: 0.59000
Epoch: [20] [   0/  20] time: 117.2867, loss: 0.01513486
Epoch: [20] [   1/  20] time: 117.5322, loss: 0.00366119
Epoch: [20] [   2/  20] time: 117.7788, loss: 0.00370290
Epoch: [20] [   3/  20] time: 118.0314, loss: 0.00026200
Epoch: [20] [   4/  20] time: 118.2929, loss: 0.00155962
Epoch: [20] [   5/  20] time: 118.5367, loss: 0.00248996
Epoch: [20] [   6/  20] time: 118.7821, loss: 0.00490251
Epoch: [20] [   7/  20] time: 119.0365, loss: 0.00164497
Epoch: [20] [   8/  20] time: 119.2965, loss: 0.00040871
Epoch: [20] [   9/  20] time: 119.5625, loss: 0.01051712
Epoch: [20] [  10/  20] time: 119.8075, loss: 0.00001184
Epoch: [20] [  11/  20] time: 120.0569, loss: 0.00076759
Epoch: [20] [  12/  20] time: 120.

Epoch: [26] [  14/  20] time: 156.6380, loss: 0.00012145
Epoch: [26] [  15/  20] time: 156.8829, loss: 0.00000252
Epoch: [26] [  16/  20] time: 157.1235, loss: 0.00001797
Epoch: [26] [  17/  20] time: 157.3797, loss: 0.00000177
Epoch: [26] [  18/  20] time: 157.6378, loss: 0.00007596
Epoch: [26] [  19/  20] time: 157.8828, loss: 0.00005842
[26/80] - ptime: 5.4107 loss: 0.00012612 acc: 0.60000
Epoch: [27] [   0/  20] time: 159.0956, loss: 0.00007041
Epoch: [27] [   1/  20] time: 159.3479, loss: 0.00005011
Epoch: [27] [   2/  20] time: 159.6115, loss: 0.00010358
Epoch: [27] [   3/  20] time: 159.8582, loss: 0.00000665
Epoch: [27] [   4/  20] time: 160.1042, loss: 0.00009915
Epoch: [27] [   5/  20] time: 160.3554, loss: 0.00000673
Epoch: [27] [   6/  20] time: 160.6167, loss: 0.00006047
Epoch: [27] [   7/  20] time: 160.8614, loss: 0.00054746
Epoch: [27] [   8/  20] time: 161.1079, loss: 0.00003459
Epoch: [27] [   9/  20] time: 161.3580, loss: 0.00000996
Epoch: [27] [  10/  20] time: 161.

...

Epoch: [47] [   8/  20] time: 282.0837, loss: 0.00001570
Epoch: [47] [   9/  20] time: 282.3364, loss: 0.00000718
Epoch: [47] [  10/  20] time: 282.5975, loss: 0.00000190
Epoch: [47] [  11/  20] time: 282.8431, loss: 0.00001768
Epoch: [47] [  12/  20] time: 283.0890, loss: 0.00001552
Epoch: [47] [  13/  20] time: 283.3431, loss: 0.00001791
Epoch: [47] [  14/  20] time: 283.6051, loss: 0.00000496
Epoch: [47] [  15/  20] time: 283.8509, loss: 0.00004185
Epoch: [47] [  16/  20] time: 284.0981, loss: 0.00000125
Epoch: [47] [  17/  20] time: 284.3557, loss: 0.00005991
Epoch: [47] [  18/  20] time: 284.6138, loss: 0.00000725
Epoch: [47] [  19/  20] time: 284.8611, loss: 0.00000411
[47/80] - ptime: 5.4555 loss: 0.00002105 acc: 0.57000
Epoch: [48] [   0/  20] time: 286.0808, loss: 0.00000053
Epoch: [48] [   1/  20] time: 286.3333, loss: 0.00000265
Epoch: [48] [   2/  20] time: 286.5921, loss: 0.00000249
Epoch: [48] [   3/  20] time: 286.8511, loss: 0.00001058
Epoch: [48] [   4/  20] time: 287.

Epoch: [54] [   6/  20] time: 323.8405, loss: 0.00006332
Epoch: [54] [   7/  20] time: 324.1033, loss: 0.00000324
Epoch: [54] [   8/  20] time: 324.3499, loss: 0.00000018
Epoch: [54] [   9/  20] time: 324.6012, loss: 0.00000463
Epoch: [54] [  10/  20] time: 324.8547, loss: 0.00003492
Epoch: [54] [  11/  20] time: 325.1103, loss: 0.00000393
Epoch: [54] [  12/  20] time: 325.3581, loss: 0.00001993
Epoch: [54] [  13/  20] time: 325.6090, loss: 0.00001999
Epoch: [54] [  14/  20] time: 325.8660, loss: 0.00000397
Epoch: [54] [  15/  20] time: 326.1205, loss: 0.00000198
Epoch: [54] [  16/  20] time: 326.3682, loss: 0.00000010
Epoch: [54] [  17/  20] time: 326.6329, loss: 0.00002997
Epoch: [54] [  18/  20] time: 326.8962, loss: 0.00001308
Epoch: [54] [  19/  20] time: 327.1611, loss: 0.00000387
[54/80] - ptime: 5.4732 loss: 0.00001249 acc: 0.58000
Epoch: [55] [   0/  20] time: 328.4178, loss: 0.00031145
Epoch: [55] [   1/  20] time: 328.6662, loss: 0.00001449
Epoch: [55] [   2/  20] time: 328.

Epoch: [61] [   4/  20] time: 365.7035, loss: 0.00000346
Epoch: [61] [   5/  20] time: 365.9529, loss: 0.00002846
Epoch: [61] [   6/  20] time: 366.2154, loss: 0.00003646
Epoch: [61] [   7/  20] time: 366.4622, loss: 0.00000690
Epoch: [61] [   8/  20] time: 366.7087, loss: 0.00000180
Epoch: [61] [   9/  20] time: 366.9586, loss: 0.00000419
Epoch: [61] [  10/  20] time: 367.2177, loss: 0.00000080
Epoch: [61] [  11/  20] time: 367.4648, loss: 0.00000037
Epoch: [61] [  12/  20] time: 367.7121, loss: 0.00000654
Epoch: [61] [  13/  20] time: 367.9636, loss: 0.00001932
Epoch: [61] [  14/  20] time: 368.2155, loss: 0.00000045
Epoch: [61] [  15/  20] time: 368.4617, loss: 0.00000033
Epoch: [61] [  16/  20] time: 368.7124, loss: 0.00000127
Epoch: [61] [  17/  20] time: 368.9600, loss: 0.00001895
Epoch: [61] [  18/  20] time: 369.2200, loss: 0.00000126
Epoch: [61] [  19/  20] time: 369.4668, loss: 0.00000084
[61/80] - ptime: 5.3915 loss: 0.00000892 acc: 0.59000
Epoch: [62] [   0/  20] time: 370.

Epoch: [68] [   2/  20] time: 407.2333, loss: 0.00002845
Epoch: [68] [   3/  20] time: 407.4897, loss: 0.00000020
Epoch: [68] [   4/  20] time: 407.7428, loss: 0.00002109
Epoch: [68] [   5/  20] time: 407.9900, loss: 0.00000096
Epoch: [68] [   6/  20] time: 408.2405, loss: 0.00000001
Epoch: [68] [   7/  20] time: 408.4998, loss: 0.00000264
Epoch: [68] [   8/  20] time: 408.7493, loss: 0.00000968
Epoch: [68] [   9/  20] time: 408.9957, loss: 0.00000026
Epoch: [68] [  10/  20] time: 409.2455, loss: 0.00000499
Epoch: [68] [  11/  20] time: 409.5079, loss: 0.00002972
Epoch: [68] [  12/  20] time: 409.7555, loss: 0.00000045
Epoch: [68] [  13/  20] time: 410.0017, loss: 0.00000021
Epoch: [68] [  14/  20] time: 410.2549, loss: 0.00000855
Epoch: [68] [  15/  20] time: 410.5113, loss: 0.00000069
Epoch: [68] [  16/  20] time: 410.7604, loss: 0.00000008
Epoch: [68] [  17/  20] time: 411.0075, loss: 0.00000159
Epoch: [68] [  18/  20] time: 411.2569, loss: 0.00000372
Epoch: [68] [  19/  20] time: 4

Epoch: [75] [   0/  20] time: 449.1733, loss: 0.00000057
Epoch: [75] [   1/  20] time: 449.4242, loss: 0.00000756
Epoch: [75] [   2/  20] time: 449.6742, loss: 0.00000761
Epoch: [75] [   3/  20] time: 449.9291, loss: 0.00000176
Epoch: [75] [   4/  20] time: 450.1762, loss: 0.00001429
Epoch: [75] [   5/  20] time: 450.4232, loss: 0.00000011
Epoch: [75] [   6/  20] time: 450.6753, loss: 0.00000726
Epoch: [75] [   7/  20] time: 450.9330, loss: 0.00000214
Epoch: [75] [   8/  20] time: 451.1801, loss: 0.00000010
Epoch: [75] [   9/  20] time: 451.4270, loss: 0.00000219
Epoch: [75] [  10/  20] time: 451.6772, loss: 0.00000074
Epoch: [75] [  11/  20] time: 451.9347, loss: 0.00001433
Epoch: [75] [  12/  20] time: 452.1813, loss: 0.00000019
Epoch: [75] [  13/  20] time: 452.4247, loss: 0.00000519
Epoch: [75] [  14/  20] time: 452.6775, loss: 0.00000146
Epoch: [75] [  15/  20] time: 452.9415, loss: 0.00000245
Epoch: [75] [  16/  20] time: 453.1869, loss: 0.00000303
Epoch: [75] [  17/  20] time: 4

In [7]:
import pygame

file='/home/huiqy/Music/CloudMusic/All_Time_Low.mp3' #文件名是完整路径名
pygame.mixer.init() #初始化音频
track = pygame.mixer.music.load(file)#载入音乐文件
pygame.mixer.music.play()#开始播放
time.sleep(60)#播放10秒
pygame.mixer.music.stop()#停止播放